In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import lxml

In [2]:
response = requests.get('http://www.nflweather.com/en/week/2010/week-1-2/')

In [3]:
weather_soup = BeautifulSoup(response.content, 'html.parser')

In [5]:
weather_soup.find_all('td')#, class_='details text-center footable-last-column')
test_tbl = weather_soup.find('table')
pd.read_html(str(test_tbl))[0]

,Unnamed: 0,Away,Game,Game.1,Game.2,Home,Time (ET),TV,Unnamed: 8,Forecast,Extended Forecast,Wind,Unnamed: 12
0,NaN,Vikings,NaN,@,NaN,Saints,Final: 9 - 14,NaN,NaN,DOME,NaN,NaN,Details
1,NaN,Dolphins,NaN,@,NaN,Bills,Final: 15 - 10,NaN,NaN,64f Overcast,NaN,NaN,Details
2,NaN,Colts,NaN,@,NaN,Texans,Final: 24 - 34,NaN,NaN,88f Partly Cloudy,NaN,NaN,Details
3,NaN,Lions,NaN,@,NaN,Bears,Final: 14 - 19,NaN,NaN,73f Fair,NaN,NaN,Details
4,NaN,Falcons,NaN,@,NaN,Steelers,Final: 9 - 15,NaN,NaN,64f Overcast,NaN,NaN,Details
5,NaN,Browns,NaN,@,NaN,Buccaneers,Final: 14 - 17,NaN,NaN,87f A Few Clouds,NaN,NaN,Details
6,NaN,Bengals,NaN,@,NaN,Patriots,Final: 24 - 38,NaN,NaN,63f A Few Clouds,NaN,NaN,Details
7,NaN,Broncos,NaN,@,NaN,Jaguars,Final: 17 - 24,NaN,NaN,91f Partly Cloudy,NaN,NaN,Details
8,NaN,Raiders,NaN,@,NaN,Titans,Final: 13 - 38,NaN,NaN,76f A Few Clouds,NaN,NaN,Details
9,NaN,Panthers,NaN,@,NaN,Giants,Final: 18 - 31,NaN,NaN,65f Light Rain,NaN,NaN,Details
